In [1]:
import re

In [2]:
with open('./Dump20200110.sql', mode='r', encoding='utf-8') as f:
    txt = f.read()

In [71]:
{'databases':[{'name':'db1', 'tables':[{'name':'tb11'}]}, {'name':'db2', 'tables':[{'name':'tb21'}]}]}

{'databases': [{'name': 'db1', 'tables': [{'name': 'tb11'}]},
  {'name': 'db2', 'tables': [{'name': 'tb21'}]}]}

In [101]:
print(sql_create_tables)

 /*!40100 DEFAULT CHARACTER SET utf8 */;

USE `gps`;

--
-- Table structure for table `atividade`
--

DROP TABLE IF EXISTS `atividade`;
/*!40101 SET @saved_cs_client     = @@character_set_client */;
/*!40101 SET character_set_client = utf8 */;
CREATE TABLE `atividade` (
  `PK_ATIV` int(11) NOT NULL AUTO_INCREMENT,
  `ATIV_DT_CADASTRO` date NOT NULL,
  `ATIV_IN_PESO` int(1) NOT NULL,
  `ATIV_IN_ORDEM` int(2) NOT NULL,
  `ATIV_DT_INICIO` date DEFAULT NULL,
  `ATIV_DT_FIM` date DEFAULT NULL,
  `ATIV_TX_NOME` varchar(100) NOT NULL,
  `ATIV_TX_DESCRICAO` varchar(250) DEFAULT NULL,
  `FK_ATIV_PROJ` int(11) NOT NULL,
  `FK_ATIV_USUA` int(11) NOT NULL,
  PRIMARY KEY (`PK_ATIV`),
  KEY `FK_ATIV_PROJ` (`FK_ATIV_PROJ`),
  CONSTRAINT `atividade_ibfk_2` FOREIGN KEY (`FK_ATIV_PROJ`) REFERENCES `projeto` (`PK_PROJ`) ON DELETE CASCADE ON UPDATE NO ACTION
) ENGINE=InnoDB AUTO_INCREMENT=289 DEFAULT CHARSET=utf8;
/*!40101 SET character_set_client = @saved_cs_client */;

--
-- Table structure for table `a

In [109]:
sql

<re.Match object; span=(832, 880), match='CREATE DATABASE /*!32312 IF NOT EXISTS*/ `cdcol`'>

In [115]:
mysql = {'databases':[]}
schemas_re = list(re.finditer(r'CREATE DATABASE.+`(.+)`', txt, re.MULTILINE))
#txt[schemas_re[0].end():]
for i in range(len(schemas_re)):
    database_name = schemas_re[i].groups()[0]
    database = {'name':database_name, 'tables':[]}
    start_sql_db = schemas_re[i].end()
    j = i+1
    if j != len(schemas_re): 
        end_sql_db = schemas_re[j].end()
    else:
        end_sql_db = len(txt)
    sql_create_tables = txt[start_sql_db:end_sql_db]
    
    tables_re = list(re.finditer(r'CREATE TABLE.+`(.+)`', sql_create_tables, re.MULTILINE))
    for k in range(len(tables_re)):
        table_name = tables_re[k].groups()[0]
        table = {'name':table_name}
        start_sql_table = schemas_re[i].end()
        l = k+1
        if l != len(tables_re): 
            end_sql_table = tables_re[l].end()
        else:
            end_sql_table = len(txt)
        sql_columns = sql_create_tables[start_sql_table:end_sql_table]

        columns_re = list(re.finditer(r'CREATE TABLE.+`(.+)`', sql_create_tables, re.MULTILINE))
        for k in range(len(tables_re)):
            table_name = tables_re[k].groups()[0]
            table = {'name':table_name}
            start_sql_table = schemas_re[k].end()
            l = k+1
            if l != len(tables_re): 
                end_sql_table = tables_re[l].end()
            else:
                end_sql_table = len(txt)
            sql_columns = sql_create_tables[start_sql_table:end_sql_table]

            database['tables'].append(table)
        print(sql_columns)
        break

    mysql['databases'].append(database)
#mysql    

IndexError: list index out of range

In [114]:
start_sql_table:end_sql_table

In [116]:
sql_create_tables

' /*!40100 DEFAULT CHARACTER SET utf8 */;\n\nUSE `safs`;\n\n--\n-- Table structure for table `acompanhamento`\n--\n\nDROP TABLE IF EXISTS `acompanhamento`;\n/*!40101 SET @saved_cs_client     = @@character_set_client */;\n/*!40101 SET character_set_client = utf8 */;\nCREATE TABLE `acompanhamento` (\n  `acom_sq_acompanhamento` int(11) NOT NULL AUTO_INCREMENT COMMENT \'PK acompanhamento\',\n  `acom_tx_nome` varchar(255) NOT NULL DEFAULT \'0\' COMMENT \'Descrição do check\',\n  `acom_nm_posicao` tinyint(1) NOT NULL DEFAULT \'0\' COMMENT \'Campo para armazenar a posição do acompanhamento.\',\n  `acom_cd_status` tinyint(1) NOT NULL DEFAULT \'1\' COMMENT \'Status do acompanhamento\',\n  `tiac_sq_tipo_acompanhamento` int(11) NOT NULL DEFAULT \'0\' COMMENT \'FK de tipo acompanhamento\',\n  PRIMARY KEY (`acom_sq_acompanhamento`),\n  KEY `FK_acompanhamento_tipo_acompanhamento` (`tiac_sq_tipo_acompanhamento`),\n  CONSTRAINT `FK_acompanhamento_tipo_acompanhamento` FOREIGN KEY (`tiac_sq_tipo_acompan